# Code to attempt to use the rechunker package

The zarr arrays were created using the scripts in `../makezarr/*.py` 

Load the data using the zarr arrays:

In [1]:
import xarray as xr

In [6]:
import zarr

In [2]:
import rechunker

In [4]:
datadir = '/scratch/tidaldrift/hycom_zarr/'
d1 = xr.open_zarr(datadir + 'hycom12-001.zarr',consolidated=False)
d1

<xarray.Dataset>
Dimensions:    (Depth: 2, Y: 7055, X: 9000, time: 455)
Coordinates:
  * Depth      (Depth) float32 0.0 15.0
  * X          (X) int32 1 2 3 4 5 6 7 8 ... 8994 8995 8996 8997 8998 8999 9000
  * Y          (Y) int32 1 2 3 4 5 6 7 8 ... 7049 7050 7051 7052 7053 7054 7055
  * time       (time) datetime64[ns] 2014-01-01T01:00:28.800000 ... 2014-01-1...
Data variables:
    Latitude   (Y, X) float32 dask.array<chunksize=(7055, 9000), meta=np.ndarray>
    Longitude  (Y, X) float32 dask.array<chunksize=(7055, 9000), meta=np.ndarray>
    u          (time, Depth, Y, X) float32 dask.array<chunksize=(1, 2, 7055, 9000), meta=np.ndarray>
    v          (time, Depth, Y, X) float32 dask.array<chunksize=(1, 2, 7055, 9000), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    experiment:   10.2
    history:      archv2ncdf3z
    source:       HYCOM archive file
    title:        HYCOM

In [7]:
d1.u.data

,Array,Chunk
Bytes,215.25 GiB,484.43 MiB
Shape,"(455, 2, 7055, 9000)","(1, 2, 7055, 9000)"
Count,2 Graph Layers,455 Chunks
Type,float32,numpy.ndarray


In [10]:
source_group = zarr.open(datadir + 'hycom12-001.zarr')
print(source_group.tree())

/
 ├── Depth (2,) float32
 ├── Latitude (7055, 9000) float32
 ├── Longitude (7055, 9000) float32
 ├── X (9000,) int32
 ├── Y (7055,) int32
 ├── time (455,) float64
 ├── u (455, 2, 7055, 9000) float32
 └── v (455, 2, 7055, 9000) float32


In [11]:
source_array = source_group['u']
source_array.info

Name,/u
Type,zarr.core.Array
Data type,float32
Shape,"(455, 2, 7055, 9000)"
Chunk shape,"(1, 2, 7055, 9000)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,231121800000 (215.2G)
No. bytes stored,111179947397 (103.5G)


In [19]:
target_chunks = (455, 1, 1,9000)
max_mem = '1000MB'

In [20]:
target_store = datadir+'hycom12-001_u_rechunked.zarr'
temp_store = datadir+'hycom12-001_u_rechunked-tmp.zarr'
array_plan = rechunker.rechunk(source_array, target_chunks, max_mem, target_store, temp_store=temp_store)
array_plan

<Rechunked>
* Source      : <zarr.core.Array '/u' (455, 2, 7055, 9000) float32>

* Intermediate: <zarr.core.Array (455, 2, 7055, 9000) float32>

* Target      : <zarr.core.Array (455, 2, 7055, 9000) float32>

In [21]:
from dask.distributed import Client, LocalCluster, progress

cluster = LocalCluster()
client = Client(cluster)

In [22]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 160,Total memory: 283.06 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45571,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 160
Started: Just now,Total memory: 283.06 GiB
Comm: tcp://127.0.0.1:42885,Total threads: 10
Dashboard: http://127.0.0.1:39441/status,Memory: 17.69 GiB
Nanny: tcp://127.0.0.1:43705,


In [23]:
from dask.diagnostics import ProgressBar

In [25]:
with ProgressBar():
    array_plan.execute()

/home/selipot/miniconda3/envs/research/lib/python3.10/site-packages/distributed/worker.py:2938: UserWarning: Large object of size 7.24 MiB detected in task graph: 
  ("('copy_read_to_intermediate-81aabf5fc78957daeb0b ... 63e2', 22905)")
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(
Task exception was never retrieved
future: <Task finished name='Task-23360' coro=<WebSocketProtocol13.write_message.<locals>.wrapper() done, defined at /home/selipot/miniconda3/envs/research/lib/python3.10/site-packages/tornado/websocket.py:1100> exception=WebSocketClosedError()>
Traceback (most recent call last):
  File "/home/selipot/miniconda3/envs/research/lib/python3.10/site-packages/tornado/websocket.py", line 1102, in wrapper
    await fut
tornado.iostre

In [45]:
d1r = zarr.open('/scratch/tidaldrift/hycom_zarr/hycom12-001_u_rechunked.zarr')

In [54]:
d1r.info

Type,zarr.core.Array
Data type,float32
Shape,"(455, 2, 7055, 9000)"
Chunk shape,"(455, 1, 1, 9000)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,231121800000 (215.2G)
No. bytes stored,110394236631 (102.8G)
Storage ratio,2.1


In [79]:
d1x = xr.open_zarr('/scratch/tidaldrift/hycom_zarr/hycom12-001_u_rechunked.zarr')

ContainsArrayError: path '' contains an array